In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp
from scipy.stats import binned_statistic
import matplotlib.pylab as plt
%matplotlib inline
from rubin_sim.utils import ddf_locations

In [11]:
# 100 potential time slots
times = np.arange(100)
m = gp.Model("try_sched")

schedule = m.addMVar(times.size*2, vtype=GRB.BINARY, name="schedule_1")

s1 = np.zeros(times.size*2)
s1[0:100] = 1
s2 = np.zeros(times.size*2)
s2[100:] = 1

mask = np.zeros(times.size*2)

# They can't be at the same time. Wow, this really slows it down.
# m.addConstr(schedule[0:100] @ schedule[100:] == 0)


# Set some random times that can't be scheduled for each
np.random.seed(42)
mask[np.round(np.random.uniform(low=0,high=99, size=20)).astype(int)] = 1
mask[np.round(np.random.uniform(low=0,high=99, size=20)).astype(int)+times.size] = 1

m.addConstr(schedule @ mask == 0)

# Limit the total number of times each one can be scheduled
n_limit_1 = 30
n_limit_2 = 30
m.addConstr(schedule @ s1 == n_limit_1)
m.addConstr(schedule @ s2 == n_limit_2)

# I want both to be as evenly distributed as possible. So, minimize chi^2
# for the difference between the cumulative distribution and an ideal cumulative
desired_cumulative_1 = np.round(times/times.max() * n_limit_1)
desired_cumulative_2 = np.round(times/times.max() * n_limit_2)

cumulative_sched = m.addMVar(times.size*2, vtype=GRB.CONTINUOUS)
cumulative_diff = m.addMVar(times.size*2, vtype=GRB.CONTINUOUS, lb=-n_limit_1, ub=n_limit_1)


m.addConstr(cumulative_sched[0] == schedule[0])
m.addConstr(cumulative_sched[100] == schedule[100])

offset = times.size
for i in np.arange(1,times.size):
    m.addConstr(cumulative_sched[i] == cumulative_sched[i-1]+schedule[i])
    m.addConstr(cumulative_diff[i] == cumulative_sched[i] - desired_cumulative_1[i])
   
    m.addConstr(cumulative_sched[i+offset] == cumulative_sched[i-1+offset]+schedule[i+offset])
    m.addConstr(cumulative_diff[i+offset] == cumulative_sched[i+offset] - desired_cumulative_2[i])
   

In [12]:
m.setObjective(cumulative_diff@cumulative_diff, GRB.MINIMIZE)

In [13]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 401 rows, 600 columns and 1232 nonzeros
Model fingerprint: 0x6ed087f4
Model has 200 quadratic objective terms
Variable types: 400 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+01]
Presolve removed 277 rows and 396 columns
Presolve time: 0.04s
Presolved: 124 rows, 204 columns, 475 nonzeros
Presolved model has 99 quadratic objective terms
Variable types: 0 continuous, 204 integer (80 binary)
Found heuristic solution: objective 64.0000000

Root relaxation: objective 7.166667e+00, 415 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [5]:
schedule.X

array([ 0.,  0.,  0.,  1., -0.,  1.,  0., -0., -0.,  1., -0., -0.,  1.,
       -0.,  0.,  0.,  1., -0.,  0.,  1., -0.,  0.,  1., -0., -0.,  1.,
       -0., -0.,  1.,  0.,  0., -0.,  1., -0., -0.,  1., -0.,  0.,  1.,
       -0., -0., -0.,  1.,  0., -0.,  1., -0., -0.,  1., -0., -0.,  0.,
        0.,  1., -0.,  1., -0., -0.,  1.,  0.,  0., -0.,  1., -0., -0.,
        1., -0., -0.,  1., -0.,  0.,  1.,  0., -0., -0.,  1., -0., -0.,
        1., -0., -0.,  1.,  0., -0., -0.,  1.,  0., -0.,  1., -0., -0.,
        1., -0., -0.,  0.,  1.,  0., -0.,  1., -0.,  0.,  0.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  1.,  0., -0.,  1.,  0.,  0.,  1., -0.,
        0., -0.,  1.,  0., -0.,  1., -0., -0.,  1.,  0.,  0.,  1.,  0.,
        0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -0.,  1.,
        0.,  0.,  0.,  1.,  0.,  1., -0., -0.,  0.,  1.,  0.,  0.,  1.,
        0., -0.,  1.,  0.,  0.,  0.,  1.,  0., -0.,  1.,  0.,  0.,  0.,
        1., -0.,  1., -0.,  0.,  0.,  1., -0.,  0.,  0.,  1.,  0